# Avance 2: Modelo relacional

In [ ]:
# Imports y configuración inicial
import pandas as pd
import numpy as np
from pathlib import Path

# para que SQLAlchemy pueda crear la base de datos SQLite
from sqlalchemy import create_engine

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

In [45]:
# Rutas y carga de datos creadas en Avance_1
project_root = Path('../')  # se va un nivel arriba desde el notebook ubicado en Avance_2 hasta la raíz del proyecto o donde estén los datos 
data_path = project_root / 'Avance_1' / 'ventasTransformed.csv'

# Carga
# Nota: el archivo CSV utiliza encoding latin1 (acentos y caracteres especiales)
df = pd.read_csv(
    data_path,
    dtype=str,
    encoding='latin1',
    sep=';' # separador utilizado en mi CSV generado por la exportación desde Power BI
)
df.shape, list(df.columns) # Verifico mi carga correcta




((30000, 33),
 ['VentaID',
  'FechaVenta',
  'HoraVenta',
  'NombreSucursal',
  'CiudadSucursal',
  'NombreVendedor',
  'NombreCliente',
  'GeneroCliente',
  'EdadCliente',
  'EmailCliente',
  'TelefonoCliente',
  'DireccionCliente',
  'MetodoPago',
  'NombreProducto1',
  'MarcaProducto1',
  'CantidadProducto1',
  'PrecioUnitarioProducto1',
  'SubtotalProducto1',
  'NombreProducto2',
  'MarcaProducto2',
  'CantidadProducto2',
  'PrecioUnitarioProducto2',
  'SubtotalProducto2',
  'NombreProducto3',
  'MarcaProducto3',
  'CantidadProducto3',
  'PrecioUnitarioProducto3',
  'SubtotalProducto3',
  'DescuentoVenta',
  'TotalVenta',
  'AñoVenta',
  'MesVenta',
  'DiaVenta'])

In [46]:
# Exploración inicial del DataFrame cargado para entender su estructura y contenido, en particular el tipo de datos y la presencia de valores nulos
df.info()
df.isna().sum().sort_values(ascending=False) # Verifico valores nulos por columna y los ordeno de mayor a menor
df.head(3) # Reviso las primeras filas para entender los datos que contiene el DataFrame


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   VentaID                  30000 non-null  object
 1   FechaVenta               30000 non-null  object
 2   HoraVenta                30000 non-null  object
 3   NombreSucursal           30000 non-null  object
 4   CiudadSucursal           30000 non-null  object
 5   NombreVendedor           30000 non-null  object
 6   NombreCliente            30000 non-null  object
 7   GeneroCliente            30000 non-null  object
 8   EdadCliente              30000 non-null  object
 9   EmailCliente             30000 non-null  object
 10  TelefonoCliente          30000 non-null  object
 11  DireccionCliente         30000 non-null  object
 12  MetodoPago               30000 non-null  object
 13  NombreProducto1          30000 non-null  object
 14  MarcaProducto1           30000 non-nul

,VentaID,FechaVenta,HoraVenta,NombreSucursal,CiudadSucursal,NombreVendedor,NombreCliente,GeneroCliente,EdadCliente,EmailCliente,TelefonoCliente,DireccionCliente,MetodoPago,NombreProducto1,MarcaProducto1,CantidadProducto1,PrecioUnitarioProducto1,SubtotalProducto1,NombreProducto2,MarcaProducto2,CantidadProducto2,PrecioUnitarioProducto2,SubtotalProducto2,NombreProducto3,MarcaProducto3,CantidadProducto3,PrecioUnitarioProducto3,SubtotalProducto3,DescuentoVenta,TotalVenta,AñoVenta,MesVenta,DiaVenta
0,113,15/12/2017,10:02:23,Techcore Medellín #1,Medellín,Liliana Jordá Armengol,Alicia Del Gras,F,33,alicia52@yahoo.com,+34 922188967,Cll 39 #21-73,Tarjeta Crédito,Hp Spectre X360,HP,2,5200000,10400000,No especificado,No especificado,0,0,0,No especificado,No especificado,0,0,0,0,104000000,2017,12,15
1,243,5/3/2024,11:04:28,Techcore Medellín #1,Medellín,Liliana Jordá Armengol,Paco Becerra Calderon,M,20,paco79@yahoo.com,+34976 312 798,Cra 10 #69-9,Tarjeta Crédito,Lenovo Yoga 7I,Lenovo,1,4400000,4400000,No especificado,No especificado,0,0,0,No especificado,No especificado,0,0,0,0,44000000,2024,3,5
2,581,17/9/2023,06:57:54,Techcore Medellín #1,Medellín,Liliana Jordá Armengol,Pascual Erasmo Pastor Izquierdo,M,29,pascual28@gmail.com,+34 900 74 14 77,Cra 88 #54-1,Tarjeta Crédito,Samsung Galaxy Book Odyssey,Samsung,1,6800000,6800000,No especificado,No especificado,0,0,0,No especificado,No especificado,0,0,0,0,68000000,2023,9,17


In [47]:
# Preparar normalización de productos. Aquí se asume que hay hasta 3 productos por venta 

product_blocks = [
    ('NombreProducto1', 'MarcaProducto1', 'CantidadProducto1', 'PrecioUnitarioProducto1', 'SubtotalProducto1'),
    ('NombreProducto2', 'MarcaProducto2', 'CantidadProducto2', 'PrecioUnitarioProducto2', 'SubtotalProducto2'),
    ('NombreProducto3', 'MarcaProducto3', 'CantidadProducto3', 'PrecioUnitarioProducto3', 'SubtotalProducto3'),
]
detalle_rows = [] # esta lista almacenará las filas normalizadas de detalle de productos (venta-producto y cantidad vendida)

In [48]:
# Crear filas de DetalleFacturas

for _, row in df.iterrows():
    venta_id = int(row['VentaID']) # asumo que VentaID es único y puede servir como clave primaria (lo veriqué en la exploración inicial)

    for (nombre, marca, cantidad, precio, subtotal) in product_blocks: 
        qty = pd.to_numeric(row[cantidad], errors='coerce') # convierto a numérico así puedo filtrar cantidades no válidas

        if qty and qty > 0: # solo agrego si hay una cantidad válida y mayor a 0
            detalle_rows.append({ 
                'FacturaID': venta_id, 
                'NombreProducto': row[nombre],
                'MarcaProducto': row[marca],
                'Cantidad': int(qty),
                'PrecioUnitario': float(row[precio]),
                'Subtotal': float(row[subtotal])
            })


In [49]:
# DataFrame final de DetalleFacturas

detalle_df = pd.DataFrame(detalle_rows) # creo el DataFrame a partir de las filas normalizadas

detalle_df.shape  # Verifico la cantidad de filas y columnas del DataFrame de detalle de productos por venta
detalle_df.head(5) # Verifico la creación correcta del DataFrame de detalle de productos por venta


,FacturaID,NombreProducto,MarcaProducto,Cantidad,PrecioUnitario,Subtotal
0,113,Hp Spectre X360,HP,2,5200000.0,10400000.0
1,243,Lenovo Yoga 7I,Lenovo,1,4400000.0,4400000.0
2,581,Samsung Galaxy Book Odyssey,Samsung,1,6800000.0,6800000.0
3,594,Samsung Galaxy Chromebook 2,Samsung,2,2000000.0,4000000.0
4,599,Hp Omen 16,HP,1,6000000.0,6000000.0


In [50]:
detalle_df['FacturaID'].nunique(), df['VentaID'].nunique() # Verifico que todas las ventas tengan al menos un producto en el detalle


(30000, 30000)

In [51]:
detalle_df.isna().sum() # Verifico si hay valores nulos en el DataFrame de detalle de productos


FacturaID         0
NombreProducto    0
MarcaProducto     0
Cantidad          0
PrecioUnitario    0
Subtotal          0
dtype: int64

Se normalizaron las columnas de productos (Producto1, Producto2 y Producto3) transformándolas en un esquema de filas.
Cada registro de la tabla DetalleFacturas representa un producto vendido dentro de una factura.

In [52]:
# Cración de tabla Productos
productos_df = (
    detalle_df[['NombreProducto', 'MarcaProducto', 'PrecioUnitario']] # selecciono solo las columnas relevantes para productos
    .drop_duplicates() # elimino duplicados para tener solo productos únicos
    .reset_index(drop=True) # reseteo el índice para tener un índice limpio
)

productos_df['ProductoID'] = [
    f'P-{str(i+1).zfill(5)}' for i in range(len(productos_df)) # creo IDs únicos para cada producto con formato P-00001, P-00002, etc.
]

productos_df = productos_df[
    ['ProductoID', 'NombreProducto', 'MarcaProducto', 'PrecioUnitario'] # reordeno columnas para mejor presentación
]

productos_df.shape
productos_df.head()


,ProductoID,NombreProducto,MarcaProducto,PrecioUnitario
0,P-00001,Hp Spectre X360,HP,5200000.0
1,P-00002,Lenovo Yoga 7I,Lenovo,4400000.0
2,P-00003,Samsung Galaxy Book Odyssey,Samsung,6800000.0
3,P-00004,Samsung Galaxy Chromebook 2,Samsung,2000000.0
4,P-00005,Hp Omen 16,HP,6000000.0


In [53]:
# Asocio ProductoID a DetalleFacturas

detalle_df = detalle_df.merge(
    productos_df, 
    on=['NombreProducto', 'MarcaProducto', 'PrecioUnitario'], # hago el merge para traer el ProductoID correspondiente
    how='left' # uso left join para mantener todas las filas de detalle_df
)

detalle_df.head()


,FacturaID,NombreProducto,MarcaProducto,Cantidad,PrecioUnitario,Subtotal,ProductoID
0,113,Hp Spectre X360,HP,2,5200000.0,10400000.0,P-00001
1,243,Lenovo Yoga 7I,Lenovo,1,4400000.0,4400000.0,P-00002
2,581,Samsung Galaxy Book Odyssey,Samsung,1,6800000.0,6800000.0,P-00003
3,594,Samsung Galaxy Chromebook 2,Samsung,2,2000000.0,4000000.0,P-00004
4,599,Hp Omen 16,HP,1,6000000.0,6000000.0,P-00005


In [54]:
# DetalleFacturas en formato relacional limpio

detalle_df = detalle_df[
    ['FacturaID', 'ProductoID', 'Cantidad', 'PrecioUnitario', 'Subtotal'] # dejo solo las columnas necesarias para DetalleFacturas
]

detalle_df.head()


,FacturaID,ProductoID,Cantidad,PrecioUnitario,Subtotal
0,113,P-00001,2,5200000.0,10400000.0
1,243,P-00002,1,4400000.0,4400000.0
2,581,P-00003,1,6800000.0,6800000.0
3,594,P-00004,2,2000000.0,4000000.0
4,599,P-00005,1,6000000.0,6000000.0


In [55]:
# Validaciones
# Todos los productos deben tener ProductoID
detalle_df['ProductoID'].isna().sum() # Verifico si hay productos sin ProductoID asignado, debe ser 0


0

In [56]:
# Cantidad de productos únicos
productos_df['ProductoID'].nunique(), productos_df.shape[0] # Verifico que no haya productos duplicados en la tabla Productos, debe ser igual al número de filas de productos_df


(80, 80)

In [57]:
# Creación de la tabla Clientes

clientes_df = (
    df[
        [
            'NombreCliente',
            'GeneroCliente',
            'EdadCliente',
            'EmailCliente',
            'TelefonoCliente',
            'DireccionCliente'
        ] 
    ] 
    .drop_duplicates() 
    .reset_index(drop=True)
)

clientes_df.shape
clientes_df.head()


,NombreCliente,GeneroCliente,EdadCliente,EmailCliente,TelefonoCliente,DireccionCliente
0,Alicia Del Gras,F,33,alicia52@yahoo.com,+34 922188967,Cll 39 #21-73
1,Paco Becerra Calderon,M,20,paco79@yahoo.com,+34976 312 798,Cra 10 #69-9
2,Pascual Erasmo Pastor Izquierdo,M,29,pascual28@gmail.com,+34 900 74 14 77,Cra 88 #54-1
3,Francisco Vega-Alcalá,M,48,francisco50@yahoo.com,+34803 330 496,Cll 3 #38-4
4,Elvira Rocamora Díaz,F,39,elvira59@hotmail.com,+34 878243990,Cll 34 #87-45


In [58]:
# Creación del ClienteID

clientes_df['ClienteID'] = [
    f'C-{str(i+1).zfill(6)}' for i in range(len(clientes_df))
]

clientes_df = clientes_df[
    [
        'ClienteID',
        'NombreCliente',
        'GeneroCliente',
        'EdadCliente',
        'EmailCliente',
        'TelefonoCliente',
        'DireccionCliente'
    ]
]

clientes_df.head()


,ClienteID,NombreCliente,GeneroCliente,EdadCliente,EmailCliente,TelefonoCliente,DireccionCliente
0,C-000001,Alicia Del Gras,F,33,alicia52@yahoo.com,+34 922188967,Cll 39 #21-73
1,C-000002,Paco Becerra Calderon,M,20,paco79@yahoo.com,+34976 312 798,Cra 10 #69-9
2,C-000003,Pascual Erasmo Pastor Izquierdo,M,29,pascual28@gmail.com,+34 900 74 14 77,Cra 88 #54-1
3,C-000004,Francisco Vega-Alcalá,M,48,francisco50@yahoo.com,+34803 330 496,Cll 3 #38-4
4,C-000005,Elvira Rocamora Díaz,F,39,elvira59@hotmail.com,+34 878243990,Cll 34 #87-45


In [59]:
# Asociar ClienteID al dataset principal

df = df.merge(
    clientes_df,
    on=[
        'NombreCliente',
        'GeneroCliente',
        'EdadCliente',
        'EmailCliente',
        'TelefonoCliente',
        'DireccionCliente'
    ],
    how='left'
)

df[['ClienteID', 'NombreCliente']].head()


,ClienteID,NombreCliente
0,C-000001,Alicia Del Gras
1,C-000002,Paco Becerra Calderon
2,C-000003,Pascual Erasmo Pastor Izquierdo
3,C-000004,Francisco Vega-Alcalá
4,C-000005,Elvira Rocamora Díaz


In [60]:
# Ninguna venta debe quedar sin ClienteID (debe dar 0)
df['ClienteID'].isna().sum()


0

Se creó la tabla Clientes deduplicando registros a partir de la información personal.
Se asignó una clave primaria ClienteID y se relacionó con la tabla Facturas.

In [61]:
df.shape

(30000, 34)

In [62]:
# Creación tabla sucursales
# Objetivo:
#   - Extraer las sucursales únicas del dataset principal
#   - Crear una tabla dimensional "Sucursales"
#   - Asignar una clave primaria (SucursalID)
#   - Preparar esta tabla para relacionarla con Facturas

# solo las columnas relacionadas con sucursales
sucursales_df = (
    df[['NombreSucursal', 'CiudadSucursal']] 
    .drop_duplicates()
        .reset_index(drop=True) # reseteamos el índice para tener un índice limpio
)

# clave primaria SucursalID
# formato: S-0001, S-0002, etc.
sucursales_df['SucursalID'] = [
    f'S-{str(i+1).zfill(4)}' for i in range(len(sucursales_df))
]

# reordenar las columnas para una presentación clara
sucursales_df = sucursales_df[
    ['SucursalID', 'NombreSucursal', 'CiudadSucursal']
]

sucursales_df.shape
sucursales_df.head()


,SucursalID,NombreSucursal,CiudadSucursal
0,S-0001,Techcore Medellín #1,Medellín
1,S-0002,Techcore Medellín #2,Medellín
2,S-0003,Techcore Bogotá #2,Bogotá
3,S-0004,Techcore Bogotá #1,Bogotá
4,S-0005,Techcore Cali,Cali


In [63]:
# Cración tabla vendedores

# 1. Seleccionamos la columna relacionada con vendedores
vendedores_df = (
    df[['NombreVendedor']]
    .drop_duplicates()
    .reset_index(drop=True)
)

# clave primaria VendedorID
# formato: V-0001, V-0002, etc.
vendedores_df['VendedorID'] = [
    f'V-{str(i+1).zfill(4)}' for i in range(len(vendedores_df))
]
# reordenamiento
vendedores_df = vendedores_df[
    ['VendedorID', 'NombreVendedor']
]

vendedores_df.shape
vendedores_df.head()

,VendedorID,NombreVendedor
0,V-0001,Liliana Jordá Armengol
1,V-0002,Ibán López Córdoba
2,V-0003,Eusebia Jara Macias
3,V-0004,Dominga Del Alcántara
4,V-0005,Ana Sofía Llopis Blázquez


In [64]:
# Verificamos que no existan IDs duplicados en Sucursales
sucursales_df['SucursalID'].nunique() == sucursales_df.shape[0]

# Verificamos que no existan IDs duplicados en Vendedores
vendedores_df['VendedorID'].nunique() == vendedores_df.shape[0]

# ambos deben ser "True"

True

In [65]:
# Mapeo de clientes

# Creamos diccionario NombreCliente y ClienteID
cliente_map = dict(zip(clientes_df['NombreCliente'], clientes_df['ClienteID']))

# Agregamos ClienteID a la tabla principal
df['ClienteID'] = df['NombreCliente'].map(cliente_map)

# Verificación rápida
df[['NombreCliente', 'ClienteID']].head()


,NombreCliente,ClienteID
0,Alicia Del Gras,C-000001
1,Paco Becerra Calderon,C-000002
2,Pascual Erasmo Pastor Izquierdo,C-000003
3,Francisco Vega-Alcalá,C-000004
4,Elvira Rocamora Díaz,C-000005


In [66]:
# Diccionario NombreSucursal - SucursalID
sucursal_map = dict(zip(sucursales_df['NombreSucursal'], sucursales_df['SucursalID']))

# Agregamos SucursalID
df['SucursalID'] = df['NombreSucursal'].map(sucursal_map)

df[['NombreSucursal', 'SucursalID']].head()


,NombreSucursal,SucursalID
0,Techcore Medellín #1,S-0009
1,Techcore Medellín #1,S-0009
2,Techcore Medellín #1,S-0009
3,Techcore Medellín #1,S-0009
4,Techcore Medellín #1,S-0009


In [67]:
# Diccionario NombreVendedor - VendedorID
vendedor_map = dict(zip(vendedores_df['NombreVendedor'], vendedores_df['VendedorID']))

# Agregamos VendedorID
df['VendedorID'] = df['NombreVendedor'].map(vendedor_map)

df[['NombreVendedor', 'VendedorID']].head()


,NombreVendedor,VendedorID
0,Liliana Jordá Armengol,V-0001
1,Liliana Jordá Armengol,V-0001
2,Liliana Jordá Armengol,V-0001
3,Liliana Jordá Armengol,V-0001
4,Liliana Jordá Armengol,V-0001


In [68]:
# Diccionario (NombreProducto, MarcaProducto, PrecioUnitario) - ProductoID
producto_map = {
    (row['NombreProducto'], row['MarcaProducto'], row['PrecioUnitario']): row['ProductoID']
    for _, row in productos_df.iterrows()
}


In [69]:
detalle_df.columns

Index(['FacturaID', 'ProductoID', 'Cantidad', 'PrecioUnitario', 'Subtotal'], dtype='object')

In [70]:
# Construcción de facturas_df

facturas_df = df[[
    'VentaID',
    'ClienteID',
    'SucursalID',
    'VendedorID',
    'MetodoPago',
    'TotalVenta',
    'DescuentoVenta',
    'FechaVenta',
    'AñoVenta',
    'MesVenta',
    'DiaVenta'
]].copy()

# Renombramos para consistencia semántica
facturas_df = facturas_df.rename(columns={'VentaID': 'FacturaID'})


In [71]:
facturas_df.shape
facturas_df.head()

,FacturaID,ClienteID,SucursalID,VendedorID,MetodoPago,TotalVenta,DescuentoVenta,FechaVenta,AñoVenta,MesVenta,DiaVenta
0,113,C-000001,S-0009,V-0001,Tarjeta Crédito,104000000,0,15/12/2017,2017,12,15
1,243,C-000002,S-0009,V-0001,Tarjeta Crédito,44000000,0,5/3/2024,2024,3,5
2,581,C-000003,S-0009,V-0001,Tarjeta Crédito,68000000,0,17/9/2023,2023,9,17
3,594,C-000004,S-0009,V-0001,Tarjeta Crédito,40000000,0,9/12/2014,2014,12,9
4,599,C-000005,S-0009,V-0001,Tarjeta Crédito,60000000,0,4/5/2023,2023,5,4


In [72]:
facturas_df[['ClienteID', 'SucursalID', 'VendedorID']].isna().sum()
# debe dar 0 las tres

ClienteID     0
SucursalID    0
VendedorID    0
dtype: int64

In [73]:
# Total ventas por marca
ventas_por_marca = (
    detalle_df
    .merge(productos_df, on='ProductoID')
    .groupby('MarcaProducto', as_index=False)['Subtotal']
    .sum()
    .sort_values('Subtotal', ascending=False)
)
ventas_por_marca


,MarcaProducto,Subtotal
5,Lenovo,6.561656e+11
4,HP,5.298100e+11
3,Dell,4.761466e+11
1,Apple,4.225344e+11
2,Asus,1.829414e+11
0,Acer,1.217888e+11
9,Samsung,6.759600e+10
6,MSI,4.775440e+10
7,Microsoft,4.519360e+10
8,Razer,2.227480e+10


###### Los números grandes se explican por el valor de venta elevado (con muchos ceros) del precio unitario.

Ajuste de métricas: Venta en millones.
Esto facilita la lectura de los valores

In [74]:
# Creación de columna en millones
ventas_por_marca['Ventas_Millones'] = ventas_por_marca['Subtotal'] / 1_000_000

In [75]:
#Redondeo
ventas_por_marca['Ventas_Millones'] = ventas_por_marca['Ventas_Millones'].round(2)

In [76]:
ventas_por_marca = ventas_por_marca.sort_values(
    'Ventas_Millones', ascending=False
)
ventas_por_marca[['MarcaProducto', 'Ventas_Millones']]

,MarcaProducto,Ventas_Millones
5,Lenovo,656165.6
4,HP,529810.0
3,Dell,476146.6
1,Apple,422534.4
2,Asus,182941.4
0,Acer,121788.8
9,Samsung,67596.0
6,MSI,47754.4
7,Microsoft,45193.6
8,Razer,22274.8


In [77]:
#Top 10 productos más vendidos

top_10_productos = (
    detalle_df
    .merge(productos_df, on='ProductoID')
    .groupby(['NombreProducto', 'MarcaProducto'], as_index=False)['Cantidad']
    .sum()
    .sort_values('Cantidad', ascending=False)
    .head(10)
)
top_10_productos

,NombreProducto,MarcaProducto,Cantidad
29,HP Spectre x360,HP,37360
36,Lenovo Legion 5 Pro,Lenovo,32033
37,Lenovo ThinkPad X1 Carbon,Lenovo,29900
40,Lenovo Yoga 7i,Lenovo,26170
27,HP Omen 16,HP,25300
34,Lenovo IdeaPad 5,Lenovo,25090
23,Dell Latitude 7420,Dell,24910
24,Dell XPS 13,Dell,22540
28,HP Pavilion 15,HP,20510
22,Dell Inspiron 15,Dell,19347


Exportación

In [78]:
# Verificación de tablas
print(facturas_df.shape)
print(detalle_df.shape)
print(productos_df.shape)
print(clientes_df.shape)
print(sucursales_df.shape)
print(vendedores_df.shape)

(30000, 11)
(60059, 5)
(80, 4)
(17453, 7)
(12, 3)
(30, 2)


In [79]:
# Ruta de salida del archivo Excel
output_path = 'modeloVentas.xlsx'

# Escritura del archivo Excel con múltiples hojas
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    
    facturas_df.to_excel(
        writer,
        sheet_name='Facturas',
        index=False
    )
    
    detalle_df.to_excel(
        writer,
        sheet_name='DetalleFacturas',
        index=False
    )
    
    productos_df.to_excel(
        writer,
        sheet_name='Productos',
        index=False
    )
    
    clientes_df.to_excel(
        writer,
        sheet_name='Clientes',
        index=False
    )
    
    sucursales_df.to_excel(
        writer,
        sheet_name='Sucursales',
        index=False
    )
    
    vendedores_df.to_excel(
        writer,
        sheet_name='Vendedores',
        index=False
    )

print("Archivo modeloVentas.xlsx exportado correctamente")

Archivo modeloVentas.xlsx exportado correctamente
